### 10.1 文本预处理技术
### 搭建简单的情感分析系统  
#### 文本预处理 
#### 1. 首先是分词，这是最基本的第一步。
无论对于英文文本，还是中文文本都离不开分词。英文的分词相对比较简单，因为一般的英文写法里通过空格来隔开不同单词的。但对于中文，我们不得不采用一些算法去做分词。   

**常用分词工具：**
- [jieba](https://github.com/fxsjy/jieba)（结巴分词） 免费使用
- HanLP（汉语言处理包） 免费使用
- SnowNLP（中文的类库） 免费使用
- FoolNLTK（中文处理工具包） 免费使用
- Jiagu（甲骨NLP） 免费使用
- pyltp（哈工大语言云） 商用需要付费
- THULAC（清华中文词法分析工具包） 商用需要付费
- NLPIR（汉语分词系统） 付费使用  

其中最常用的还是Jieba(结巴)分词，又快又有效。

In [ ]:
# encoding=utf-8
import jieba

# 基于jieba的分词  参考： https://github.com/fxsjy/jieba
seg_list = jieba.cut("贪心学院是国内最专业的人工智能在线教育品牌", cut_all=False)
#seg_list = jieba.cut("小明硕士毕业于中国科学院计算所，后在日本早稻田大学深造", cut_all=False) # cut_all=False 精确模式
print("Default Mode: " + "/ ".join(seg_list))  

# TODO: 在jieba中加入"贪心学院"关键词, hint: 通过.add_word函数
jieba.add_word("贪心学院")
seg_list = jieba.cut("贪心学院是国内最专业的人工智能在线教育品牌", cut_all=False)
print("Default Mode: " + "/ ".join(seg_list)) 

一般情况下，我们还是要定义属于自己的专有名词的。如果我们考虑的是医疗领域，则需要把医疗领域我们比较关注的词先加入到词库里，再通过结巴工具做分词，毕竟很多的专有词汇并不存在于结巴的词库里。  
大部分情况下只需要使用工具去分词就可以了，没必要自己造轮子。但有一些特殊情况，比如这些开源工具的效果很一般，或者它们缺少某些方面的考虑，则可能需要自己写一个**分词工具**。  
实际上，自己写一个分词工具也不难，目前常用的方法是基于**HMM, CRF**等方法。  

#### 2. 单词的过滤或者字符的过滤。
比如把一些出现次数特别多的单词过滤掉也叫作**停用词的过滤**，或者把那些出现次数特别少的单词过滤掉，或者把一些特殊符号比如#@过滤掉。  
**停用词: 就是那些出现特别频繁，但对于一个句子贡献不是特别大的单词。** 比如”的“， ”他“可以认为是停用词。英文里头"the","an","their"等。  
使用停用词的方法也超级简单。就是提前设计好停用词库，然后做文本分析时把这些停用词忽略掉就可以了。  

##### 2.1 停用词库的构建可以有三种方法:
- 第一、手动去设置停用词库，把所有的停用词写入一个文件。这个过程比较耗费时间，但对于非常垂直类的应用还是最有效的。
- 第二、从网上搜索停用词库，一般来讲网络上可以找到大部分语言的停用词库，这些都是别人已经整理好的，所以基本都是通用的。但有些时候确实由于应用本身的特点，这些停用词库可能还满足不了需求。所以，这时候需要适当地加入一些人工方式来整理的单词。
- 第二、从网上搜索停用词库，一般来讲网络上可以找到大部分语言的停用词库，这些都是别人已经整理好的，所以基本都是通用的。但有些时候确实由于应用本身的特点，这些停用词库可能还满足不了需求。所以，这时候需要适当地加入一些人工方式来整理的单词。




#### 下面以NLTK为例来学习一下如何使用停用词库。

In [ ]:
# 以NLTK为例来学习一下如何使用停用词库。
# 方法一：自己建立一个停用词词典
stop_words = ["the","an","is","students"]
# 在使用时：假设word_list包含了文本里的单词
word_list = ["we","are","the","students"]
filtered_words = [word for word in word_list if word not in stop_words]
print(filtered_words)

# 方法二： 直接利用别人已经构好的停用词库
from nltk.corpus import stopwords
cachedStopWords = stopwords.words("english")
print(cachedStopWords)


##### 2.2 过滤出现频率低的单词
一般来讲，比如一个单词出现少于10次或者20次，我们可以归类为是可以去掉的单词。 但这个也取决于手里的语料库大小。如果语料库本身总共只包含了不到一千个单词，那这个阈值显然有点高了。  
对于特殊符号，我们也需要做一些处理。特殊符号其实就是我们觉得不太有用的符号。比如一个文章里出现的@#&，这些可以认为是特殊符号，进而可以去掉。

##### 2.3 文本中的数字
首先，数字本身是有意义的，至少说明这个是一个“数字“， 但具体是什么数字其实很难把它表示出来。为什么呢？ 因为数字不像单词有一个完整的可以提前定义好的库。数字本身是无穷多的，我们没有办法把所有的都列出来。而且具体是什么数对于理解文本来说意义没有那么大。

基于这些理由，我们通常把出现的所有的数表示成一个统一的特殊符号比如"#NUM"，这样至少我们的模型知道这是一个数字。

##### 2.4 英文文本
对于英文文本，我们通常会做单词标准化的操作，也就是把类似含义的单词统一表示成一种形式。这里有两种常用的方法，分别是**stemming和lemmazation。**
